In [20]:
# overlay variant information(support matrix) with scRNA matrix

In [21]:
library(Seurat)
support_mtx = Seurat::Read10X('/SGRNJ06/randd/USER/zhouyiqi/2024/test/snp1.5.1/support_mtx',gene.column = 1)

In [22]:
# convert the matrix to a dataframe
df = as.data.frame(t(support_mtx))
head(df)

,variant_1 1 114590381 A T,variant_2 1 114707695 T C,variant_3 1 114708169 T A,variant_4 1 114713817 C T,variant_5 1 114713826 C T,variant_6 1 114713846 A G,variant_7 1 114713852 A G,variant_8 1 114713859 G C,variant_9 1 114713875 A G,variant_10 1 114713880 T C,⋯,variant_197 7 55174791 T C,variant_198 7 55174796 G A,variant_199 7 55174796 G T,variant_200 7 55174797 A G,variant_201 7 55174807 A G,variant_202 7 55181309 C T,variant_203 7 55181347 A G,variant_204 7 55181370 G A,variant_205 7 55181378 C T,variant_206 7 55191822 T G
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAACATCGAACAACCAACAGATTC,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACATCGAACGCTTAAGATCGCA,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACATCGAACGCTTAGCTAACGA,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACATCGAACGTGATCAAGGAGC,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACATCGAACGTGATTGAAGAGA,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACATCGAAGACGGAGCCAAGAC,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [37]:
# Pull a variant of interest, for example variant 20
library(dplyr)
library(stringr)
gt20 = df['variant_20 1 114713964 T C']
gt20 <- gt20 %>%
  rename(`chr1:114713964:T-C` = `variant_20 1 114713964 T C`)
gt20$CID = 1:nrow(gt20)
head(gt20)

,chr1:114713964:T-C,CID
,<dbl>,<int>
AAACATCGAACAACCAACAGATTC,0,1
AAACATCGAACGCTTAAGATCGCA,1,2
AAACATCGAACGCTTAGCTAACGA,1,3
AAACATCGAACGTGATCAAGGAGC,1,4
AAACATCGAACGTGATTGAAGAGA,0,5
AAACATCGAAGACGGAGCCAAGAC,1,6


In [38]:
# read raw count
raw_count = read.table('/SGRNJ06/randd/USER/zhouyiqi/2024/test/snp1.5.1/D0722_2_13NARS_3_FJ_variant_count.tsv', header=TRUE)
count20 = raw_count %>% filter(VID==20)
head(count20)

,VID,CID,ref_count,alt_count
,<int>,<int>,<int>,<int>
1,20,2,249,0
2,20,3,1,0
3,20,4,208,0
4,20,6,231,0
5,20,8,35,0
6,20,9,111,0


In [48]:
# merge by cell ID
x = gt20
y = count20
merge20 <- merge(x, y, by.x = "CID", by.y = "CID", all.x = TRUE)
merge20$VID = NULL
merge20[is.na(merge20)] <- 0
row.names(merge20) = row.names(gt20)
head(merge20)

,CID,chr1:114713964:T-C,ref_count,alt_count
,<int>,<dbl>,<dbl>,<dbl>
AAACATCGAACAACCAACAGATTC,1,0,0,0
AAACATCGAACGCTTAAGATCGCA,2,1,249,0
AAACATCGAACGCTTAGCTAACGA,3,1,1,0
AAACATCGAACGTGATCAAGGAGC,4,1,208,0
AAACATCGAACGTGATTGAAGAGA,5,0,0,0
AAACATCGAAGACGGAGCCAAGAC,6,1,231,0


In [49]:
# rename number to meaningful label
merge20 <- merge20 %>%
  mutate(`chr1:114713964:T-C` = recode(as.character(`chr1:114713964:T-C`), 
                       `0` = "No Call", 
                       `1` = "ref/ref", 
                       `2` = "alt/alt", 
                       `3` = "alt/ref"))
head(merge20)

,CID,chr1:114713964:T-C,ref_count,alt_count
,<int>,<chr>,<dbl>,<dbl>
AAACATCGAACAACCAACAGATTC,1,No Call,0,0
AAACATCGAACGCTTAAGATCGCA,2,ref/ref,249,0
AAACATCGAACGCTTAGCTAACGA,3,ref/ref,1,0
AAACATCGAACGTGATCAAGGAGC,4,ref/ref,208,0
AAACATCGAACGTGATTGAAGAGA,5,No Call,0,0
AAACATCGAAGACGGAGCCAAGAC,6,ref/ref,231,0


In [50]:
# merge with scRNA matrix, add the variant to metadata
seurat.data <- Read10X("/SGRNJ06/randd/USER/zhouyiqi/2024/test/snp1.5.1/scRNA_matrix")
seurat_obj <- CreateSeuratObject(seurat.data, min.cells = 1, project = "seurat_obj")
seurat_obj <- AddMetaData(object = seurat_obj, metadata = merge20)
head(seurat_obj[[]])

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


,orig.ident,nCount_RNA,nFeature_RNA,CID,chr1:114713964:T-C,ref_count,alt_count
,<fct>,<dbl>,<int>,<int>,<chr>,<dbl>,<dbl>
AAACATCGAACAACCAACAGATTC,seurat_obj,786,529,1,No Call,0,0
AAACATCGAACGCTTAAGATCGCA,seurat_obj,415,261,2,ref/ref,249,0
AAACATCGAACGCTTAGCTAACGA,seurat_obj,812,460,3,ref/ref,1,0
AAACATCGAACGTGATCAAGGAGC,seurat_obj,597,349,4,ref/ref,208,0
AAACATCGAACGTGATTGAAGAGA,seurat_obj,819,576,5,No Call,0,0
AAACATCGAAGACGGAGCCAAGAC,seurat_obj,859,548,6,ref/ref,231,0
